# Import Libraries and Tools and Create Helper Functions

In [1]:
# Requirements: pandas, numpy, scikit-learn, xgboost, joblib, matplotlib
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

# Models for scaled data
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Tree based models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
integrase = pd.read_csv('../data/ii_df_clean.csv')
integrase.head()

,molecule_chembl_id,canonical_smiles,standard_type,standard_value,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_units_norm,IC50_nM,pIC50
0,CHEMBL304722,O=C(CC(O)CCc1ccc(O)c(O)c1)O[C@H]1Cc2cc(O)c(O)c...,IC50,1400.0,5.85,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,nM,1400.0,5.853872
1,CHEMBL304722,O=C(CC(O)CCc1ccc(O)c(O)c1)O[C@H]1Cc2cc(O)c(O)c...,IC50,1000.0,6.00,CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,nM,1000.0,6.000000
2,CHEMBL67076,O=C(CCc1ccc(O)c(O)c1)c1ccc(O)c(O)c1O,IC50,1700.0,5.77,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,nM,1700.0,5.769551
3,CHEMBL67076,O=C(CCc1ccc(O)c(O)c1)c1ccc(O)c(O)c1O,IC50,1000.0,6.00,CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,nM,1000.0,6.000000
4,CHEMBL177126,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H]1[C@H](O)C[C@](O...,IC50,250.0,6.60,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,nM,250.0,6.602060


# Data Preprocessing

We have appended the morgan fingerprints and molecular descriptors into a single dataframe

In [3]:
df = pd.read_csv('../data/ii_qsar_features.csv')
print("rows, cols:", df.shape)

rows, cols: (6386, 523)


In [4]:
df.head()

,MolWt,MolLogP,MolMR,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,NumAromaticRings,HeavyAtomCount,FractionCSP3,...,Morgan_512_503,Morgan_512_504,Morgan_512_505,Morgan_512_506,Morgan_512_507,Morgan_512_508,Morgan_512_509,Morgan_512_510,Morgan_512_511,pIC50
0,498.484,2.8920,125.2126,177.14,7,10,7,3,36,0.269231,...,0,1,0,0,1,0,0,0,0,5.853872
1,498.484,2.8920,125.2126,177.14,7,10,7,3,36,0.269231,...,0,1,0,0,1,0,0,0,0,6.000000
2,290.271,2.0301,73.8595,118.22,5,6,4,2,21,0.133333,...,0,0,0,0,0,0,0,0,0,5.769551
3,290.271,2.0301,73.8595,118.22,5,6,4,2,21,0.133333,...,0,0,0,0,0,0,0,0,0,6.000000
4,516.455,1.0296,125.1976,211.28,7,11,7,2,37,0.240000,...,0,0,1,0,1,0,0,0,0,6.602060


In [5]:
# Prepare X and y
y = df['pIC50']
X = df.drop(columns=['pIC50'])
print('Feature count: ', X.shape[1])

Feature count:  522


In [6]:
y.isna().sum()

np.int64(0)

In [7]:
# ------------------------------------------------------------------------------
# Quick feature cleanup
# ------------------------------------------------------------------------------

# a. remove near-constant features

# Fit the selector
vt = VarianceThreshold(threshold=1e-6)
X_v = vt.fit_transform(X)
# Features kept
kept_features = X.columns[vt.get_support()].tolist()
# Features removed
removed_features = [col for col in X.columns if col not in kept_features]
# Create the reduced DataFrame
X = pd.DataFrame(X_v, columns=kept_features)

print(f"Features after variance filter: {len(kept_features)}")
print(f"Features removed: {len(removed_features)}")
# Optional: display or save the removed features
print("\nRemoved features (first 20):")
print(removed_features[:20])

Features after variance filter: 522
Features removed: 0

Removed features (first 20):
[]


In [8]:
# b. (optional) remove extremely collinear features (simple correlation filter)

corr_thresh = 0.98
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > corr_thresh)]
if to_drop:
    X.drop(columns=to_drop, inplace=True)
    print("Dropped highly correlated features:", to_drop)

Dropped highly correlated features: ['MolMR', 'HeavyAtomCount']


In [9]:
X.shape

(6386, 520)

In [10]:
# ------------------------------------------------------------------------------
# Define descriptor columns to scale and fingerprint columns to pass through
# ------------------------------------------------------------------------------
descriptors = ['MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'FractionCSP3']
fingerprints = [c for c in X.columns if c not in descriptors]

# ColumnTransformer: scale descriptors, pass fingerprints through
preprocessor = ColumnTransformer([
    ("desc", StandardScaler(), descriptors),
    ("fp", "passthrough", fingerprints)
])

In [11]:
import os

SAVE_DIR = "../artifacts"
os.makedirs(SAVE_DIR, exist_ok=True)

# 1. Save final feature column list (after filters)
final_features = X.columns.tolist()
pd.Series(final_features).to_csv(os.path.join(SAVE_DIR, "integrase_features.csv"), index=False)

# QSAR Model Training

In [12]:
# ---------------------------
# k-fold OOF trainer
# ---------------------------
def kfold_train_predict(model, X_df, y_arr, n_splits=5, random_state=42):
    """
    model: estimator or pipeline (should accept fit/predict)
    X_df: pandas DataFrame (full dataset)
    y_arr: numpy array or Series
    returns: oof_preds (np.array same length as X_df), mean_r2, mean_rmse
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(X_df))
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_df, y_arr), 1):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y_arr[train_idx], y_arr[val_idx]

        # fit on train fold, predict on val fold
        model.fit(X_train, y_train)
        preds = model.predict(X_val)

        oof_preds[val_idx] = preds

        fold_r2 = r2_score(y_val, preds)
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_scores.append((fold_r2, fold_rmse))
        print(f"Fold {fold}: R2 = {fold_r2:.3f}, RMSE = {fold_rmse:.3f}")

    mean_r2 = np.mean([s[0] for s in fold_scores])
    mean_rmse = np.mean([s[1] for s in fold_scores])
    print(f"\nMean CV R2 = {mean_r2:.3f}, Mean RMSE = {mean_rmse:.3f}")
    return oof_preds, mean_r2, mean_rmse

In [13]:
# ---------------------------
# Define models and pipelines
# ---------------------------
models = {}

# Linear models (need scaling) -> include preprocessor
models['Linear'] = Pipeline([("preproc", preprocessor), ("model", LinearRegression())])
models['ElasticNet'] = Pipeline([("preproc", preprocessor), ("model", ElasticNet(random_state=42, max_iter=5000))])

# SVR (needs scaling)
models['SVM'] = Pipeline([("preproc", preprocessor), ("model", SVR())])

# KNN
models['KNN'] = Pipeline([("preproc", preprocessor), ("model", KNeighborsRegressor())])

# MLP
models['MLP'] = Pipeline([("preproc", preprocessor), ("model", MLPRegressor(max_iter=2000, random_state=42))])

# Random Forest (trees don't need scaling, but pipeline is fine)
models['Random Forest'] = Pipeline([("preproc", preprocessor), ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])

# XGBoost
models['XGBoost'] = Pipeline([("preproc", preprocessor), ("model", XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1, objective='reg:squarederror'))])

In [14]:
# ---------------------------
# Run k-fold for each model
# ---------------------------
results = {}
oof_dict = {}

for name, model in models.items():
    print(f"\n=== Training model: {name} ===")
    oof_preds, mean_r2, mean_rmse = kfold_train_predict(model, X, y, n_splits=5, random_state=42)
    results[name] = {"r2": mean_r2, "rmse": mean_rmse}
    oof_dict[name] = oof_preds

# Summary
print("\nSummary (mean CV results):")
for name, res in results.items():
    print(f"{name}: R2 = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")


=== Training model: Linear ===
Fold 1: R2 = 0.632, RMSE = 0.811
Fold 2: R2 = 0.650, RMSE = 0.818
Fold 3: R2 = 0.631, RMSE = 0.838
Fold 4: R2 = 0.618, RMSE = 0.836
Fold 5: R2 = 0.653, RMSE = 0.797

Mean CV R2 = 0.637, Mean RMSE = 0.820

=== Training model: ElasticNet ===
Fold 1: R2 = -0.001, RMSE = 1.337
Fold 2: R2 = -0.001, RMSE = 1.382
Fold 3: R2 = -0.001, RMSE = 1.380
Fold 4: R2 = -0.000, RMSE = 1.352
Fold 5: R2 = -0.000, RMSE = 1.352

Mean CV R2 = -0.001, Mean RMSE = 1.361

=== Training model: SVM ===
Fold 1: R2 = 0.732, RMSE = 0.692
Fold 2: R2 = 0.763, RMSE = 0.672
Fold 3: R2 = 0.745, RMSE = 0.697
Fold 4: R2 = 0.739, RMSE = 0.691
Fold 5: R2 = 0.761, RMSE = 0.661

Mean CV R2 = 0.748, Mean RMSE = 0.683

=== Training model: KNN ===
Fold 1: R2 = 0.709, RMSE = 0.721
Fold 2: R2 = 0.752, RMSE = 0.688
Fold 3: R2 = 0.740, RMSE = 0.703
Fold 4: R2 = 0.715, RMSE = 0.721
Fold 5: R2 = 0.763, RMSE = 0.658

Mean CV R2 = 0.736, Mean RMSE = 0.698

=== Training model: MLP ===
Fold 1: R2 = 0.643, RMS

# Tune the Best Models

### Support Vector Machine (SVR),
### K-Nearest Neighbours (KNN),
### Random Forest (RF)

In [15]:
import optuna
import joblib
from sklearn.decomposition import PCA

In [16]:
# PCA variant of the model
n_fp = len(fingerprints)
print(f"Using {len(descriptors)} descriptor columns and {n_fp} fingerprint columns.")

if n_fp == 0:
    raise ValueError("No fingerprint columns detected. Make sure your X contains fingerprint bit columns.")

# ---------- CV settings ----------
cv_outer = KFold(n_splits=5, shuffle=True, random_state=42)

Using 8 descriptor columns and 512 fingerprint columns.


In [17]:
# ----------------------- SVM ---------------------------------
def run_optuna_svm(n_trials=80):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # SVR hyperparameters (log-uniform sampling where appropriate)
        kernel = trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"])
        C = trial.suggest_loguniform("C", 1e-3, 1e3)
        epsilon = trial.suggest_loguniform("epsilon", 1e-4, 1.0)
        gamma_choice = trial.suggest_categorical("gamma_choice", ["scale", "auto", "numeric"])
        if gamma_choice == "numeric":
            gamma = trial.suggest_loguniform("gamma", 1e-5, 1e1)
        else:
            gamma = gamma_choice
        # degree and coef0 for polynomial kernel
        if kernel == "poly":
            degree = trial.suggest_int("degree", 2, 5)
            coef0 = trial.suggest_float("coef0", 0.0, 1.0)
        else:
            degree = 3
            coef0 = 0.0

        svr = SVR(kernel=kernel, C=C, epsilon=epsilon, gamma=gamma, degree=degree, coef0=coef0, max_iter=100000)
        pipe = Pipeline([("preproc", preprocessor), ("svr", svr)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    # reconstruct SVR
    best_kernel = best.get("kernel", "rbf")
    best_gamma = best.get("gamma", best.get("gamma_choice", "scale"))
    if best_gamma == "numeric":
        best_gamma = best.get("gamma")
    best_degree = best.get("degree", 3)
    best_coef0 = best.get("coef0", 0.0)

    best_svm = SVR(
        kernel=best_kernel,
        C=best.get("C"),
        epsilon=best.get("epsilon"),
        gamma=best_gamma,
        degree=best_degree,
        coef0=best_coef0,
        max_iter=100000
    )
    best_pipe = Pipeline([("preproc", preprocessor_best), ("svr", best_svm)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/ii_svm.joblib")
    print("SVM best R2:", study.best_value)
    print("SVM best params:", study.best_params)
    return study, best_pipe

In [18]:
# ------------------ KNN -----------------------
def run_optuna_knn(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        p = trial.suggest_int("p", 1, 2)  # 1 = manhattan, 2 = euclidean

        knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
        pipe = Pipeline([("preproc", preprocessor), ("knn", knn)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    best_knn = KNeighborsRegressor(n_neighbors=best["n_neighbors"], weights=best["weights"], p=best["p"])
    best_pipe = Pipeline([("preproc", preprocessor_best), ("knn", best_knn)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/ii_knn.joblib")
    print("KNN best R2:", study.best_value)
    print("KNN best params:", study.best_params)
    return study, best_pipe

In [19]:
# -------------------- Random Forest --------------------------------
def run_optuna_rf(n_trials=50):
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        n_estimators = trial.suggest_int("n_estimators", 100, 500)
        max_depth = trial.suggest_int("max_depth", 3, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5, 0.8])

        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, n_jobs=-1, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("rf", rf)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])
    best_rf = RandomForestRegressor(n_estimators=best["n_estimators"], max_depth=best["max_depth"],
                                    min_samples_split=best["min_samples_split"],
                                    min_samples_leaf=best["min_samples_leaf"],
                                    max_features=best["max_features"], n_jobs=-1, random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("rf", best_rf)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/ii_rf.joblib")
    print("RF best R2:", study.best_value)
    print("RF best params:", study.best_params)
    return study, best_pipe

In [20]:
# ----------------------------
# Example: run all studies sequentially (adjust n_trials)
# ----------------------------
svm_study, svm_pipe = run_optuna_svm(n_trials=50)

[I 2025-10-07 23:14:18,200] A new study created in memory with name: no-name-d0522039-98ec-40c8-b41e-dd23de756aac
Best trial: 0. Best value: 0.673067:   2%|▏         | 1/50 [00:26<21:20, 26.13s/it]

[I 2025-10-07 23:14:44,315] Trial 0 finished with value: 0.6730669192851183 and parameters: {'use_pca': True, 'pca_n_components': 60, 'kernel': 'poly', 'C': 17.363418028138508, 'epsilon': 0.06440092769435873, 'gamma_choice': 'numeric', 'gamma': 0.037223405208821315, 'degree': 3, 'coef0': 0.009404400250143774}. Best is trial 0 with value: 0.6730669192851183.


Best trial: 1. Best value: 0.678432:   4%|▍         | 2/50 [00:36<13:30, 16.88s/it]

[I 2025-10-07 23:14:54,739] Trial 1 finished with value: 0.678432362256722 and parameters: {'use_pca': True, 'pca_n_components': 87, 'kernel': 'poly', 'C': 0.0972913694709314, 'epsilon': 0.00037184377096643903, 'gamma_choice': 'scale', 'degree': 3, 'coef0': 0.49563748458983936}. Best is trial 1 with value: 0.678432362256722.


Best trial: 1. Best value: 0.678432:   6%|▌         | 3/50 [01:03<16:44, 21.37s/it]

...

[I 2025-10-07 23:32:01,796] Trial 47 finished with value: 0.7332095555513304 and parameters: {'use_pca': False, 'kernel': 'poly', 'C': 0.2284913475018331, 'epsilon': 0.04977444945057876, 'gamma_choice': 'scale', 'degree': 5, 'coef0': 0.7761012983832327}. Best is trial 44 with value: 0.7571860637964287.


Best trial: 44. Best value: 0.757186:  98%|█████████▊| 49/50 [17:55<00:19, 19.61s/it]

[I 2025-10-07 23:32:13,430] Trial 48 finished with value: 0.5805236940837432 and parameters: {'use_pca': False, 'kernel': 'rbf', 'C': 0.6335018392059615, 'epsilon': 0.633813780310462, 'gamma_choice': 'auto'}. Best is trial 44 with value: 0.7571860637964287.


Best trial: 44. Best value: 0.757186: 100%|██████████| 50/50 [18:03<00:00, 21.67s/it]


[I 2025-10-07 23:32:21,672] Trial 49 finished with value: -107012818.3257521 and parameters: {'use_pca': True, 'pca_n_components': 131, 'kernel': 'sigmoid', 'C': 359.18102877306717, 'epsilon': 0.17463835870650532, 'gamma_choice': 'scale'}. Best is trial 44 with value: 0.7571860637964287.
SVM best R2: 0.7571860637964287
SVM best params: {'use_pca': False, 'kernel': 'rbf', 'C': 1.9741986621252687, 'epsilon': 0.1683259141820452, 'gamma_choice': 'scale'}


In [21]:
knn_study, knn_pipe = run_optuna_knn(n_trials=50)

[I 2025-10-07 23:32:29,196] A new study created in memory with name: no-name-33fa74a9-fd5b-4639-8d61-bb0c272c51c7
Best trial: 0. Best value: 0.717428:   2%|▏         | 1/50 [00:03<02:33,  3.14s/it]

[I 2025-10-07 23:32:32,334] Trial 0 finished with value: 0.7174278616172077 and parameters: {'use_pca': True, 'pca_n_components': 223, 'n_neighbors': 5, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 0.7174278616172077.


Best trial: 0. Best value: 0.717428:   4%|▍         | 2/50 [00:04<01:32,  1.93s/it]

[I 2025-10-07 23:32:33,417] Trial 1 finished with value: 0.6827277421159136 and parameters: {'use_pca': False, 'n_neighbors': 29, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 0.7174278616172077.


Best trial: 2. Best value: 0.722721:   6%|▌         | 3/50 [00:06<01:40,  2.14s/it]

...

[I 2025-10-07 23:34:02,767] Trial 47 finished with value: 0.730641930064712 and parameters: {'use_pca': True, 'pca_n_components': 189, 'n_neighbors': 9, 'weights': 'uniform', 'p': 1}. Best is trial 21 with value: 0.7433186359426733.


Best trial: 48. Best value: 0.743577:  98%|█████████▊| 49/50 [01:34<00:01,  1.55s/it]

[I 2025-10-07 23:34:03,826] Trial 48 finished with value: 0.7435774151106533 and parameters: {'use_pca': True, 'pca_n_components': 137, 'n_neighbors': 5, 'weights': 'uniform', 'p': 2}. Best is trial 48 with value: 0.7435774151106533.


Best trial: 48. Best value: 0.743577: 100%|██████████| 50/50 [01:37<00:00,  1.94s/it]


[I 2025-10-07 23:34:06,435] Trial 49 finished with value: 0.7181139387977344 and parameters: {'use_pca': True, 'pca_n_components': 138, 'n_neighbors': 13, 'weights': 'distance', 'p': 1}. Best is trial 48 with value: 0.7435774151106533.
KNN best R2: 0.7435774151106533
KNN best params: {'use_pca': True, 'pca_n_components': 137, 'n_neighbors': 5, 'weights': 'uniform', 'p': 2}


In [22]:
rf_study, rf_pipe = run_optuna_rf(n_trials=50)

[I 2025-10-07 23:34:06,654] A new study created in memory with name: no-name-c7539331-b983-41dd-801b-01f971345083
Best trial: 0. Best value: 0.705659:   2%|▏         | 1/50 [00:24<19:54, 24.37s/it]

[I 2025-10-07 23:34:31,021] Trial 0 finished with value: 0.7056591119187683 and parameters: {'n_estimators': 357, 'max_depth': 24, 'min_samples_split': 16, 'min_samples_leaf': 13, 'max_features': 0.3}. Best is trial 0 with value: 0.7056591119187683.


Best trial: 1. Best value: 0.716178:   4%|▍         | 2/50 [00:26<08:56, 11.18s/it]

[I 2025-10-07 23:34:32,973] Trial 1 finished with value: 0.7161779591510615 and parameters: {'n_estimators': 123, 'max_depth': 45, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 1 with value: 0.7161779591510615.


Best trial: 1. Best value: 0.716178:   6%|▌         | 3/50 [00:51<13:44, 17.54s/it]

...

[I 2025-10-07 23:59:32,585] Trial 47 finished with value: 0.7485247544960054 and parameters: {'n_estimators': 240, 'max_depth': 41, 'min_samples_split': 17, 'min_samples_leaf': 3, 'max_features': 0.3}. Best is trial 35 with value: 0.7550601667384464.


Best trial: 35. Best value: 0.75506:  98%|█████████▊| 49/50 [26:00<00:31, 31.03s/it]

[I 2025-10-08 00:00:07,314] Trial 48 finished with value: 0.7462443462985651 and parameters: {'n_estimators': 333, 'max_depth': 46, 'min_samples_split': 13, 'min_samples_leaf': 5, 'max_features': 0.3}. Best is trial 35 with value: 0.7550601667384464.


Best trial: 35. Best value: 0.75506: 100%|██████████| 50/50 [26:05<00:00, 31.31s/it]


[I 2025-10-08 00:00:12,194] Trial 49 finished with value: 0.7137117673114867 and parameters: {'n_estimators': 289, 'max_depth': 39, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 35 with value: 0.7550601667384464.
RF best R2: 0.7550601667384464
RF best params: {'n_estimators': 141, 'max_depth': 41, 'min_samples_split': 12, 'min_samples_leaf': 2, 'max_features': 0.3}


# Try a meta learner or ensemble of the best models

In [23]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict

# reproducible folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 1) Get OOF predictions for each base model (these preds are made by models
#    that were trained without the corresponding sample — no leakage)
print("Generating OOF preds (this may take time)...")
oof_knn = cross_val_predict(knn_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_svm = cross_val_predict(svm_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_rf  = cross_val_predict(rf_pipe,  X, y, cv=kf, n_jobs=-1, method='predict')

# Stack OOF predictions (n_samples x n_models)
stack_oof = np.vstack([oof_knn, oof_svm, oof_rf]).T

# 2) Simple average ensemble
ens_mean = stack_oof.mean(axis=1)
r2_mean = r2_score(y, ens_mean)
rmse_mean = np.sqrt(mean_squared_error(y, ens_mean))
print(f"Simple average ensemble -> R2: {r2_mean:.4f}, RMSE: {rmse_mean:.4f}")

Generating OOF preds (this may take time)...
Simple average ensemble -> R2: 0.7710, RMSE: 0.6512


In [24]:
# 3) Stacking: train a Ridge meta-learner on the OOF stack
meta = Ridge(alpha=1.0)
meta.fit(stack_oof, y)                 # training on OOF preds is OK (no leakage)
ens_stack = meta.predict(stack_oof)    # predictions on the same OOF matrix
r2_stack = r2_score(y, ens_stack)
rmse_stack = np.sqrt(mean_squared_error(y, ens_stack))
print(f"Stacking (Ridge) ensemble -> R2: {r2_stack:.4f}, RMSE: {rmse_stack:.4f}")

# Show meta-learner weights
print("Meta-learner coefficients (weights):", meta.coef_)
print("Meta-learner intercept:", meta.intercept_)

Stacking (Ridge) ensemble -> R2: 0.7715, RMSE: 0.6505
Meta-learner coefficients (weights): [0.32157995 0.36259839 0.33838847]
Meta-learner intercept: -0.1357436691919176


In [25]:
joblib.dump(meta, "../models/ii_meta.joblib")
print("Saved final base pipelines and meta-learner.")

Saved final base pipelines and meta-learner.
